In [2]:
FILE_START_DATE ='2023-09-25'
FILE_END_DATE = '2023-12-31'

import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 100
pd.options.display.max_rows = 30

def file_to_string(fileName, startDate=FILE_START_DATE, endDate=FILE_END_DATE):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    print("StartDate "+startDate,"\nEndDate "+endDate)
    return string.replace('$STARTDATE$',startDate).replace('$ENDDATE$',endDate)

def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def run_query(query):
    print("Query Start " + timestamp())
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    print("Query End " + timestamp())
    return df
    
def add_conditionals(df):
    df = df.copy()
    df['Date']=pd.to_datetime(df['Date'])
    
    df['Proactive ExAsst Group'] = (df['Employee'].isin((
        573190, 573276, 572909, 573192, 573585,
        573573, 572815, 552121, 576911, 572247
        ))).astype(int)

    df['After Reactive  ExAsst Launch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['After Proactive ExAsst Launch']=(df['Date']>=datetime(2023,11,10)).astype(int)

    df['Has Proactive'] = df['Proactive ExAsst Group'] * df['After Proactive ExAsst Launch']
    
    df['Has Reactive'] = (df['Days Since First Expert Assist Use']>=0).astype(int)

    df['Test Group For This Period'] = np.select(
        condlist = [df['Has Proactive']==1,
                    df['Has Reactive']==1],
                    choicelist = ['Proactive','Reactive'],
                    default='Control')

    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    
    if df.columns.isin(['TaskQueue']).any():
        for queue in df['TaskQueue'].unique():
            df[queue] = (df['TaskQueue']==queue).astype(int)
    df['Constant'] =1
    return df

def add_metrics(df):
    df = df.copy()
    df['AHT'] = df['Total_Resolution_Time']/df['Sessions']
    df['HERO'] = df['HERO_XR_Score']/df['HERO_XR_Count']
    df['SP100'] = 100*df['Total_Accepted']/df['Total_Eligible']
    df['Transfers'] = df['Transfer_Count']/df['Transfer_Score']
    df['Constant'] =1
    return df

In [ ]:
df.columns.isin(['TaskQueue']).any()

In [ ]:
df_daily = run_query(file_to_string('DailyMetrics.SQL'))
df_daily.head()
df_daily['Date'] = pd.to_datetime(df_daily['Date'])

In [ ]:
metric = {'SP100':['Total_Accepted','Total_Eligible']}
metric

weekly_summary = df_daily.copy()
weekly_summary = add_conditionals(weekly_summary)
weekly_summary['Week'] = weekly_summary['Date'].dt.isocalendar().week
weekly_summary['SP100']=100*weekly_summary['Total_Accepted']/weekly_summary['Total_Eligible']
weekly_summary['After Proactive ExAsst Launch']=weekly_summary['After Proactive ExAsst Launch'].astype(bool)
px.box(weekly_summary,y='SP100',color='After Proactive ExAsst Launch')

# weekly_summary = pd.pivot_table(weekly_summary,index='Date',aggfunc=sum,columns='Test Group After Launch',values=['Total_Accepted','Total_Eligible'])
# weekly_summary=weekly_summary['Total_Accepted']/weekly_summary['Total_Eligible']
# weekly_summary = weekly_summary*100
# # weekly_summary['Proactive Average'] = weekly_summary['Proactive'].mean()
# px.line(weekly_summary,title='SP100').update_layout(yaxis_title="SP100")


In [ ]:
metricList = ['Total_Resolution_Time']
metric_Inputs = ['Sessions','Total_Resolution_Time''HERO_XR_Score','HERO_XR_Count', 'Transfer_Score', 'Transfer_Count',
              'Helix_Searches', 'Helix_Sessions' 'Total_Eligible','Total_Accepted']
test_df = add_conditionals(df_daily)
test_df = add_metrics(test_df)
px.histogram(test_df['Total_Resolution_Time']).show()
test_df = test_df.dropna()
test_df.info(verbose=False)
del test_df, metricList, metric_Inputs

In [ ]:
test_df = add_conditionals(df_daily)
test_df = add_metrics(test_df)

test_df = test_df.dropna()
# display(test_df.head(5))

display(sm.OLS(
    endog = test_df['AHT'],
    exog = test_df[['Has Proactive','Weeks Ago','Constant']]
    ).fit().summary())
del test_df

In [5]:
df_sessions = run_query(file_to_string('AHT And Sales.SQL'))
display(df_sessions['Date'].min(),df_sessions['Date'].max())

StartDate 2023-09-25 
EndDate 2023-12-31
Query Start 2024-01-17 10:11:46 AM


C:\Users\austen.moore\AppData\Local\Temp\ipykernel_20004\968787740.py:28: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query,cnxn)


Query End 2024-01-17 10:13:25 AM


datetime.date(2023, 9, 25)

datetime.date(2023, 12, 31)

In [10]:
test_df = add_conditionals(df_sessions.copy())

# test_df = test_df[test_df['Handle Time']<=5000]
# test_df['Handle Time Log'] = np.log(test_df['Handle Time'])


for queue in test_df['TaskQueue'].unique():
    test_df[queue] = (test_df['TaskQueue']==queue).astype(int)
display(test_df.columns)
display(test_df['Date'].min(),test_df['Date'].max())
X = test_df[[ 'Constant', 
            'Has Proactive',
            'Has Reactive',
            # 'After Proactive ExAsst Launch',
            'Proactive ExAsst Group',
            'Any Offer', 'Any Accept',
            'Weeks Ago',
            'VZN_MTS_Bundle_EN_2233', 'VZN_Onboarding_2628'
            ]]

mdl = sm.OLS(
    endog = test_df['Handle Time'],
    exog = X
    ).fit()
display(mdl.summary())
test_df['Prediction']=mdl.predict(X)
test_df['Resid'] = test_df['Handle Time']-test_df['Prediction']

# import scipy.stats as stats
# from matplotlib import pyplot as plt
# fig = sm.qqplot(test_df['Resid'], stats.t, distargs=(7,),line='45', fit=True)
# plt.show()

# px.histogram(test_df['Resid']).show()
# px.scatter(test_df, x='Handle Time',y='Resid', color='Has Proactive', opacity=.2).show()
# test_df.to_excel('Dataset.xlsx')
del test_df, mdl, queue, X

Index(['Date', 'Employee', 'Calls', 'TaskQueue', 'ucid',
       'Days Since First Expert Assist Use', 'Handle Time', 'SR_Hits',
       'SO_Hits', 'Any Offer', 'Any Accept', 'Proactive ExAsst Group',
       'After Reactive  ExAsst Launch', 'After Proactive ExAsst Launch',
       'Has Proactive', 'Has Reactive', 'Test Group For This Period', 'Day',
       'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday',
       'Sunday', 'Week', 'Weeks Ago', 'Week 43', 'Week 39', 'Week 45',
       'Week 44', 'Week 50', 'Week 52', 'Week 47', 'Week 48', 'Week 46',
       'Week 42', 'Week 51', 'Week 49', 'Week 40', 'Week 41',
       'VZN_5G_UPSELL_5013', 'VZN_Onboarding_2628', 'VZN_MTS_Bundle_EN_2233',
       'VZN_MTS_Bundle_SP_2234', 'VZN_SameDay_Replac_ACT_2792',
       'HZNVZN_Service_En_5019', 'HZNVZN_Security_Advisor_EN_2603',
       'VZN_ASG_To_Soluto_2485', 'VZN_Onboarding_SP_2629',
       'VZN_Resolution_SP_2462', 'HZNVZN_GAI_5084',
       'HZNVZN_Security_Advisor_SP_2604', 'Consta

Timestamp('2023-09-25 00:00:00')

Timestamp('2023-12-31 00:00:00')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            Handle Time   R-squared:                       0.059
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     8870.
Date:                Wed, 17 Jan 2024   Prob (F-statistic):               0.00
Time:                        10:19:11   Log-Likelihood:            -9.3987e+06
No. Observations:             1126411   AIC:                         1.880e+07
Df Residuals:                 1126402   BIC:                         1.880e+07
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Constant                 882.1393      4.167    211.681      0.000     873.972     890.307
Has Proactive              8.0521     23.546      0.342      0.732     -38.097      54.201
Has Reactive               0.2738      3.047      0.090      0.928      -5.698       6.245
Proactive ExAsst Group  -208.2596     17.067    -12.203      0.000    -241.710    -174.809
Any Offer                292.1017      2.042    143.024      0.000     288.099     296.105
Any Accept               649.9605      3.890    167.068      0.000     642.335     657.586
Weeks Ago                  4.2987      0.309     13.929      0.000       3.694       4.904
VZN_MTS_Bundle_EN_2233   166.8749      3.322     50.234      0.000     160.364     173.386
VZN_Onboarding_2628      451.0320      4.007    112.548      0.000     443.177     458.886
==============================================================================
Omnibus:                   686273.069   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         30321891.780
Skew:                           2.305   Prob(JB):                         0.00
Kurtosis:                      27.996   Cond. No.                         213.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""